In [1]:
import re

In [2]:
test_case = """
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
"""

expected = [
    'bright white',
    'muted yellow',
    'dark orange',
    'light red',
]

In [3]:
rules_str = test_case.strip().splitlines()
rules_str

['light red bags contain 1 bright white bag, 2 muted yellow bags.',
 'dark orange bags contain 3 bright white bags, 4 muted yellow bags.',
 'bright white bags contain 1 shiny gold bag.',
 'muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.',
 'shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.',
 'dark olive bags contain 3 faded blue bags, 4 dotted black bags.',
 'vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.',
 'faded blue bags contain no other bags.',
 'dotted black bags contain no other bags.']

In [4]:
n_color_re = re.compile(r"(?P<n>\d*)[\s]*(?P<color>[a-z\s]*) bag")

def parse_rule_str(r):
    groups = re.findall(n_color_re, r)
    
    top_color = groups[0][1]
    if top_color.endswith('bags contain no other'):
        return top_color[:-21], []
    #contains = [(int(n), c) for n, c in groups[1:]]
    contains = [c for n, c in groups[1:]]
    return top_color, contains

rules = [parse_rule_str(r) for r in rules_str]
rules = dict(rules)
rules

{'bright white': ['shiny gold'],
 'dark olive': ['faded blue', 'dotted black'],
 'dark orange': ['bright white', 'muted yellow'],
 'dotted black ': [],
 'faded blue ': [],
 'light red': ['bright white', 'muted yellow'],
 'muted yellow': ['shiny gold', 'faded blue'],
 'shiny gold': ['dark olive', 'vibrant plum'],
 'vibrant plum': ['faded blue', 'dotted black']}

In [5]:
from collections import defaultdict

# Revert dictionary
contained_to_container = defaultdict(list)
for container, all_contained in rules.items():
    for contained in all_contained:
        contained_to_container[contained].append(container)

In [6]:
contained_to_container

defaultdict(list,
            {'bright white': ['dark orange', 'light red'],
             'dark olive': ['shiny gold'],
             'dotted black': ['dark olive', 'vibrant plum'],
             'faded blue': ['dark olive', 'muted yellow', 'vibrant plum'],
             'muted yellow': ['dark orange', 'light red'],
             'shiny gold': ['muted yellow', 'bright white'],
             'vibrant plum': ['shiny gold']})

In [7]:
target = 'shiny gold'
target_containers = set(contained_to_container[target])

new_target_containers = set(target_containers)
while True:
    for c in list(target_containers):
        new_target_containers.update(contained_to_container[c])

    if target_containers == new_target_containers:
        break

    target_containers = set(new_target_containers)

In [8]:
target_containers, new_target_containers

({'bright white', 'dark orange', 'light red', 'muted yellow'},
 {'bright white', 'dark orange', 'light red', 'muted yellow'})

In [9]:
len(target_containers)

4

# Part 1

In [10]:
with open('input.txt', 'r') as f:
    rules_str = f.read().strip().splitlines()

In [11]:
len(rules_str)

594

In [12]:
rules = [parse_rule_str(r) for r in rules_str]
rules = dict(rules)
len(rules)

594

In [13]:
rules['mirrored green']

['dark brown', 'shiny gold', 'clear plum', 'faded teal']

In [14]:
# Revert dictionary
target = 'shiny gold'
rules_not_target = dict(rules)
del rules_not_target[target]

contained_to_container = defaultdict(list)
for container, all_contained in rules_not_target.items():
    for contained in all_contained:
        contained_to_container[contained].append(container)

In [15]:
target = 'shiny gold'
target_containers = set(contained_to_container[target])

new_target_containers = set(target_containers)
while True:
    for c in list(target_containers):
        new_target_containers.update(contained_to_container[c])

    if target_containers == new_target_containers:
        break

    target_containers = set(new_target_containers)

In [16]:
len(target_containers)

179

# Part 2

In [17]:
expected_n_bags = 32

In [18]:
def parse_rule_str(r):
    groups = re.findall(n_color_re, r)
    
    top_color = groups[0][1]
    if top_color.endswith('bags contain no other'):
        return top_color[:-22], []
    contains = [(c, int(n)) for n, c in groups[1:]]
    return top_color, contains



In [19]:
rules_str = test_case.strip().splitlines()
rules_str

rules = [parse_rule_str(r) for r in rules_str]
rules = dict(rules)
len(rules)

9

In [20]:
def count_inside(color, multiplier=1):
    inside = dict(rules[color])
    if len(inside) == 0:
        return multiplier
    
    tot = 0
    for c, n in inside.items():
        tot += count_inside(c, multiplier=n)
    
    return tot * multiplier + multiplier


In [21]:
target_color = 'shiny gold'
count_inside(target_color) - 1

32

In [22]:
target_color = 'shiny gold'
assert count_inside(target_color) - 1 == expected_n_bags

In [23]:
test_case_2 = """
shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.
"""

expected_n_bags_2 = 126

rules_str = test_case_2.strip().splitlines()
rules = [parse_rule_str(r) for r in rules_str]
rules = dict(rules)
len(rules)

7

In [24]:
target_color = 'shiny gold'
assert count_inside(target_color) - 1 == expected_n_bags_2

In [25]:
with open('input.txt', 'r') as f:
    rules_str = f.read().strip().splitlines()
rules = [parse_rule_str(r) for r in rules_str]
rules = dict(rules)
len(rules)

594

In [26]:
target_color = 'shiny gold'
count_inside(target_color) - 1

18925